In [1]:
%load_ext autoreload
%autoreload 2
from kadboxes.util import enable_viewer
enable_viewer("CH Water Sensor")

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
import cadquery as cq
import numpy as np
from cq_warehouse.thread import IsoThread
from cq_warehouse.fastener import SetScrew
import cq_warehouse.extensions


In [3]:
sphere_radius = 20
cylinder_radius = 11
cylinder_height = sphere_radius*2.1  # Make sure the height is enough to cut through the sphere

# Create a sphere
# sphere = cq.Solid.makeSphere(sphere_radius)
ellipse = cq.Workplane("XZ").lineTo(sphere_radius, 0).ellipseArc(sphere_radius, sphere_radius/2, angle1=0, angle2=90).close()
ellipse = ellipse.revolve(360, axisStart=(0, 0, 0), axisEnd=(0, 1, 0))
ellipse2 = ellipse.rotate((0,0,0), (0,1,0), 180)
ellipse = ellipse.union(ellipse2)
ellipse

In [4]:
# Create the central hole for the pipe
cylinder = cq.Workplane("XY").cylinder(cylinder_height, cylinder_radius)
cylinder = cylinder.translate((0, 0, sphere_radius - cylinder_height / 2))
result = ellipse.cut(cylinder)
result

In [5]:
# Create the hole for the sensor
cylinder = cq.Workplane("XZ").cylinder(cylinder_radius*1.3, 2.5).translate((0,cylinder_radius*1.5,0))
result = result.cut(cylinder)
result

In [6]:
# Screw holes (M4 = 2mm radius)
offset = cylinder_radius + ((sphere_radius - cylinder_radius) / 2)

cylinder = cq.Workplane("XZ").cylinder(40, 2)
result = result.cut(cylinder.translate((offset,0,0)))
result = result.cut(cylinder.translate((-offset,0,0)))
result

In [7]:
# Add threads
thread = IsoThread(4, 0.7, 10, external=False, end_finishes=("fade", "fade"))
thread = thread.rotate((0,0,0), (1,0,0), 90).translate((0,5,0))
result = result.union(thread.translate((offset,0,0)))
result = result.union(thread.translate((-offset,0,0)))
result

In [8]:
screw_head = 4
clipping = 17
screw_head_cylinder = cq.Workplane("XZ").cylinder(20, screw_head)
result = result.cut(screw_head_cylinder.translate((offset,-clipping ,0)))
result = result.cut(screw_head_cylinder.translate((-offset,-clipping ,0)))
result = result.cut(screw_head_cylinder.translate((offset,clipping ,0)))
result = result.cut(screw_head_cylinder.translate((-offset,clipping ,0)))
result

In [9]:
# split = result.rotate((0,0,0), (1,0,0), 90)
split = result
split = split.split(keepTop=True, keepBottom=True)
# split = split.rotate((0,0,0), (1,0,0), -90)
split

100% ⋮————————————————————————————————————————————————————————————⋮ (2/2)  0.70s


In [11]:
cq.exporters.export(split.all()[1], 'ch-water-sensor.stl')
cq.exporters.export(split.all()[0], 'ch-water-fix.stl')
